In [ ]:
# Importing Packages and Set Up

%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors
from matplotlib.mlab import bivariate_normal
from pylab import *
import numpy as np

import astropy.units as u
from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord, EarthLocation, Angle
from astropy.coordinates import FK5

import detectorgeometry as geom

In [ ]:
# Input ANITA Flight Path Data

N_sim =25
A_lat = np.asarray([]) # Earth latitude in degrees
A_long = np.asarray([])# Earth longitude in degrees
A_alt = np.asarray([])
A_ice = np.asarray([])
A_time = np.asarray([])

orbit = Time('2013-12-10T00:00:00.0') - Time('2013-12-01T00:00:00.0')
t_sim = orbit/N_sim
    
for i in range(0,N_sim):
    inc = 360/N_sim
    A_lat = np.append(A_lat, float(-75.0))
    A_long = np.append(A_long,float(i)*inc)
    A_alt = np.append(A_alt,35.0)
    A_ice = np.append(A_ice,3.0)
    A_sim = Time('2013-12-10T00:00:00.0')+i*t_sim
    A_time = np.append(A_time, A_sim.value)

h = A_alt 
lat_loc = Angle(A_lat *u.deg)
lon_loc = Angle(A_long * u.deg)
A_loc = EarthLocation(lat = lat_loc,lon = lon_loc, height = h *u.km )

radius = lambda x: np.sqrt(A_loc.geocentric[0][x]**2+A_loc.geocentric[1][x]**2+A_loc.geocentric[2][x]**2)
A_R = radius(range(0,len(A_loc)))


In [ ]:
# Map Creation Parameters
sky_bin_dec = np.linspace(-90,90,181)
sky_bins = np.tile(sky_bin_dec,(360,1))

print sky_bins.shape


In [ ]:
# Movie Creation Parameters

N_bins = 25 # number of bins dividing flight time

t = Time(A_time, format='isot', scale='utc')
t_start = min(t)# start JD of flight
t_end = max(t) # end JD of flight
t_width = (t_end - t_start)/N_bins # some interval of time that is the width of the bins

In [ ]:
# Importing Effective Areas for Various Source Angles
import csv
with open("eff_area.csv", 'rb') as csvfile:
    areas_ref = csv.reader(csvfile)

    eff_areas=[]
    for row in areas_ref:
        eff_areas.append(row[0])
    
eff_areas=np.asarray(eff_areas,dtype=float64)

In [ ]:
# determining effective area for each bin of the sky map
sky_maps_frames_sep=[]
sky_maps_frames_total=[]

sky_map_total = np.asarray([[0 for x in range(181)] for y in range(360)],dtype=float64)
for k in range(0,N_bins):
    print k
    sky_map_b = np.asarray([[0 for x in range(181)] for y in range(360)],dtype=float64 )
    # determine observations that fall into the bin
    N_obs = len(A_time)
    start_index = int(k*N_obs/N_bins)
    end_index = start_index + int(N_obs/N_bins)
    
    t_bin = Time(A_time[start_index:end_index], format='isot', scale='utc')
    t_mean = min(t_bin)+t_width

    # determine ANITA approximate position for that time bin
    A_lat_bin = np.mean(A_lat[start_index:end_index])
    A_lon_bin = np.mean(A_long[start_index:end_index])
    A_alt_bin = np.mean(A_alt[start_index:end_index])
    A_ice_bin = np.mean(A_ice[start_index:end_index])
    A_R_bin = np.mean(A_R[start_index:end_index])

    h_bin = A_alt_bin
    lat_loc_bin = Angle(A_lat_bin *u.deg)
    lon_loc_bin = Angle(A_lon_bin * u.deg)
    A_loc_bin = EarthLocation(lat = lat_loc_bin,lon = lon_loc_bin, height = h_bin *u.km )
            
    for i in range(0,360):
        # print i
        bin_ra = Angle(i * u.deg)
        bin_dec = Angle(sky_bins[i] * u.deg)
    
        # determine the altitude and azimuth of the sky map bin from ANITA's point of view
        sky_bin_loc = SkyCoord(ra = bin_ra, dec = bin_dec, frame = 'icrs', obstime = t_mean, location = A_loc_bin)
        sky_bin_alt_az = sky_bin_loc.transform_to('altaz')
        sky_bin_alt = sky_bin_alt_az.alt

        # convert from sky bin altitude to source angle
        bin_elev = sky_bin_alt.value
        theta_src = 180+bin_elev - 90
        # print bin_dec,theta_src

        A = np.interp(theta_src,np.linspace(0,89,90),eff_areas,0.0,0.0)

        #sky_map_ra=np.asarray(A)
        # print i, len(theta_src),len(A),sky_map_ra.shape
        sky_map_b[i] = np.asarray(A,dtype=float64)
        #print min(A[90:]), min(sky_map_b[i][90:])
    
    
    sky_map_total = sky_map_total+sky_map_b
    sky_maps_frames_sep.append(sky_map_b)
    
    sky_map_mean = sky_map_total/float(k+1)
    sky_maps_frames_total.append(sky_map_mean)

In [1]:
# Making Sky Map for each Bin
for k in range(0,N_bins):
    fig = plt.figure(figsize=(10,10))

    left = 0
    right = 360
    bottom = -90
    top = 90
    extent = [left, right, bottom, top]
    
#     plt.imshow(np.flip(np.transpose(sky_maps_frames_sep[k])), extent = extent)
#     plt.text(370,20,'Linear Color Scale',rotation=90,fontsize=12)
    
    plt.imshow(np.flip(np.transpose(sky_maps_frames_sep[k]),0), extent = extent, norm=matplotlib.colors.LogNorm())
    plt.text(370,20,'Log Color Scale',rotation=90,fontsize=12)
    plt.colorbar(shrink=0.5)

    plt.ylabel("Declination (degrees)", fontsize=18)
    plt.xlabel("Right Ascension (degrees)", fontsize=18)
    plt.title("ANITA Geometric Effective Area Sky Projection", fontsize=20)
    
    filename = "bin_sep"+str(k)+".png"
    plt.savefig(filename)
    plt.show()

NameError: name 'N_bins' is not defined

In [ ]:
print sky_maps_frames_sep[0][150][90:]

In [ ]:
# Making Cumulative Sky Map
for k in range(0,N_bins):
    fig = plt.figure(figsize=(10,10))

    left = 0
    right = 360
    bottom = -90
    top = 90
    extent = [left, right, bottom, top]
    
#     plt.imshow(np.flip(np.transpose(sky_maps_frames_total[k])), extent = extent)
#     plt.text(370,20,'Linear Color Scale',rotation=90,fontsize=12)
    
    plt.imshow(np.flip(np.transpose(sky_maps_frames_total[k]),0), extent = extent, norm=matplotlib.colors.LogNorm())
    plt.text(370,20,'Log Color Scale',rotation=90,fontsize=12)
    plt.colorbar(shrink=0.5)

    plt.ylabel("Declination (degrees)", fontsize=18)
    plt.xlabel("Right Ascension (degrees)", fontsize=18)
    plt.title("ANITA Geometric Effective Area Sky Projection", fontsize=20)
    filename = "bin_tot"+str(k)+".png"
    plt.savefig(filename)
    plt.show()


In [ ]:
# Making Movie
import imageio
images = []
for k in range(0,N_bins):
    filename = "bin_sep"+str(k)+".png"
    images.append(imageio.imread(filename))
imageio.mimsave('dec25_movie_sep_log.gif', images)

images = []
for k in range(0,N_bins):
    filename = "bin_tot"+str(k)+".png"
    images.append(imageio.imread(filename))
imageio.mimsave('dec25_movie_tot_log.gif', images)